In [ ]:
#!/usr/bin/env python
# coding: utf-8

<br>
<img src="resources/cropped-SummerWorkshop_Header.png">  <br>
<br>
<h1 align="center">EM Connectomics - elli</h1> <br>
<h2 align="center">Summer Workshop on the Dynamic Brain </h2> <br>
<br>
<br>
<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; "><br>
<br>
<br>
</div><br>
<br>


for functional neuron stuff...<br>
look at the projections from each interneuron type onto the functionally coregistered neurons <br>
use the ids of the coreistered neurons, maybe add another column to the df with "functional" or not, use this instead of passing in "inhib" or "excit"

BPC = VIP neurons --> don't make many synapses onto excitatory neurons<br>
<br>
MC = SST neurons --> make synapses onto dendritic branches of excitatory neurons<br>
<br>
BC = PV neurons --> makes synapses onto the soma of excitatory neurons

In[1]:

In [ ]:
from caveclient import CAVEclient
from meshparty import trimesh_io, trimesh_vtk
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
client = CAVEclient('minnie65_public_v117')

In[2]:

In [ ]:
plt.rcParams.update({'font.size': 22})
plt.rcParams["axes.labelsize"] = 24

In[3]:

In [ ]:
import pickle5 as pickle

In [ ]:
with open('/efs/ellismith/SWDB_2021/DynamicBrain/allen_cell_types.pkl', "rb") as fh:
    predicted_cell_type_df = pickle.load(fh)
    
predicted_cell_type_df= predicted_cell_type_df.rename(columns={'cell_type': 'predicted_cell_type'})

In [ ]:
predicted_cell_type_df.head()

In[39]:

In [ ]:
with open('/efs/ellismith/SWDB_2021/DynamicBrain/coreg_matches_v158_matched_v117.pkl', "rb") as fh:
    coreg_matched_df = pickle.load(fh)

In[40]:

In [ ]:
coreg_matched_df.head()

In[42]:

In [ ]:
corr_poss_df = pd.read_csv('corr_poss_df.csv')

In[45]:

In [ ]:
corr_merged_df = corr_poss_df.merge(coreg_matched_df, on=['session', 'scan_idx', 'unit_id'], how='inner')

In[46]:

In [ ]:
corr_merged_df

In[ ]:

In [ ]:
with open('/efs/ellismith/SWDB_2021/DynamicBrain/allen_cell_types.pkl', "rb") as fh:
    predicted_cell_type_df = pickle.load(fh)

In[5]:

In [ ]:
predicted_cell_type_df.shape

In[6]:

In [ ]:
client.materialize.get_tables()

In[7]:

to get the neuron and synapse tables

In [ ]:
syn_table_resolution = client.materialize.get_table_metadata(client.materialize.synapse_table)['voxel_resolution']
# client.materialize.get_tables()

In [ ]:
cln_df=client.materialize.query_table('proofreading_status_public_release', 
                              filter_in_dict={'status_axon': ['clean', 'extended']})

get all the neurons

In [ ]:
nrn_df = client.materialize.query_table('nucleus_neuron_svm',
                                        filter_equal_dict={'cell_type':'neuron'})

How many times does a root id appears in the table.<br>
Since each row is a nucleus, this effectively counts the number of neuronal somata.

In [ ]:
soma_count_df = (
    nrn_df
        .query('cell_type == "neuron"')
        .groupby('pt_root_id')
        .count()
        .rename(columns={'valid': 'num_soma'})
    )[['num_soma']]

pull out the predictions about e vs i

In [ ]:
ei_df = client.materialize.query_table('allen_soma_ei_class_model_v1')
# remove the predictions where there is more than one soma
ei_df_single = ei_df.drop_duplicates(subset=['pt_root_id'])

In[8]:

In [ ]:
ei_df_single.head()

In[9]:

In [ ]:
functional_df=client.materialize.query_table('func_unit_em_match_release')

In [ ]:
functional_ids = np.array(functional_df['pt_root_id']).tolist()

In[10]:

In [ ]:
len(functional_ids)

In[11]:

In [ ]:
pv_df = predicted_cell_type_df.loc[predicted_cell_type_df['predicted_cell_type']=='BC']
vip_df = predicted_cell_type_df.loc[predicted_cell_type_df['predicted_cell_type']=='BPC']
sst_df = predicted_cell_type_df.loc[predicted_cell_type_df['predicted_cell_type']=='MC']

In [ ]:
pv_ids = pv_df['pt_root_id'].to_list()
vip_ids = vip_df['pt_root_id'].to_list()
sst_ids = sst_df['pt_root_id'].to_list()

In[12]:

In [ ]:
def get_single_neuron_outputs(cell_id, cell_type_ei):
    
    '''
    gives outputs from a single presynaptic neuron
    args: 
    cell_id = neuron id for one neuron (int)
    returns: 
    df
    '''

    # get all the synapses from this cell
    out_df = client.materialize.synapse_query(pre_ids=cell_id)

    # merge this with the synapses with the neuron table
    # use how='left' to keep all the synapses but drop unused nuclei
    post_syn_df = out_df.merge(
        soma_count_df,
        left_on='post_pt_root_id',
        right_index=True,
        how='left',
    ).fillna(0)

    # filtering my df by num_soma
    # the presyn neuron here is pre_pt_root_id, so that's the same for the whole table
    # the postsyn neuron for each synapse is post_pt_root_id
    post_syn_df=post_syn_df.loc[post_syn_df['num_soma']==1.0]

In [ ]:
    # merge in the e/i prediction into a dataframe
    merged_df = post_syn_df.merge(
        ei_df_single[['pt_root_id', 'cell_type', 'pt_position']],
        left_on='post_pt_root_id',
        right_on='pt_root_id',
        how='left',
    ).drop(columns='pt_root_id')
    
    # merge in the cell type prediction into a dataframe
    merged_df = merged_df.merge(predicted_cell_type_df[['pt_root_id', 'predicted_cell_type', 'pt_position']],
                    left_on='post_pt_root_id',
                    right_on='pt_root_id',
                    how='left',
                    suffixes=["_ei", "_ct"]
                ).drop(columns='pt_root_id')
    
        
    #output_df= merged_df[np.isnan(merged_df.post_pt_root_id)==0]

    # calculating synapse lengths
    syn_pos = np.array(merged_df['ctr_pt_position'])
    soma_pos = np.array(merged_df['pt_position_ei'])
    syn_distance = []
        
#       syn_distance = np.linalg.norm(syn_pos - soma_pos, axis=1)

    # to convert voxels to microns
    for i in range(len(merged_df)):
        dist = np.linalg.norm(np.array([4, 4, 40]) /1000 * (syn_pos[i]-soma_pos[i]))
        syn_distance.append(dist)
    merged_df['synapse_distance'] = syn_distance
    
    merged_df = merged_df.loc[merged_df['cell_type']==cell_type_ei]
    
    return merged_df

In[13]:

In [ ]:
def get_single_neuron_inputs(cell_id, cell_type_ei):
    
    '''
    gives inputs to a single postsynaptic neuron
    args: 
    cell_id = neuron id for one neuron (int)
    returns: 
    df
    '''

    # get all the synapses from this cell
    in_df = client.materialize.synapse_query(post_ids=cell_id)

    # merge this with the synapses with the neuron table
    # use how='left' to keep all the synapses but drop unused nuclei
    pre_syn_df = in_df.merge(
        soma_count_df,
        left_on='pre_pt_root_id',
        right_index=True,
        how='left',
    ).fillna(0)

    # filtering my df by num_soma
    pre_syn_df=pre_syn_df.loc[pre_syn_df['num_soma']==1.0]

In [ ]:
    # merge in the e/i prediction into a dataframe
    merged_input_df = pre_syn_df.merge(
        ei_df_single[['pt_root_id', 'cell_type', 'pt_position']],
        left_on='pre_pt_root_id',
        right_on='pt_root_id',
        how='left',
    ).drop(columns='pt_root_id')
    
    # merge in the cell type prediction into a dataframe
    merged_input_df = merged_input_df.merge(predicted_cell_type_df[['pt_root_id', 'predicted_cell_type', 'pt_position']],
                    left_on='pre_pt_root_id',
                    right_on='pt_root_id',
                    how='left',
                    suffixes=["_ei", "_ct"]
                ).drop(columns='pt_root_id')
    
        
    #output_df= merged_df[np.isnan(merged_df.post_pt_root_id)==0]

    # calculating synapse lengths
    syn_pos = np.array(merged_input_df['ctr_pt_position'])
    soma_pos = np.array(merged_input_df['pt_position_ei'])
    syn_distance = []
        
#       syn_distance = np.linalg.norm(syn_pos - soma_pos, axis=1)

    # to convert voxels to microns
    for i in range(len(merged_input_df)):
        dist = np.linalg.norm(np.array([4, 4, 40]) /1000 * (syn_pos[i]-soma_pos[i]))
        syn_distance.append(dist)
    merged_input_df['synapse_distance'] = syn_distance
    
    merged_input_df = merged_input_df.loc[merged_input_df['cell_type']==cell_type_ei]
    
    return merged_input_df

In[14]:

In [ ]:
def get_multi_neuron_outputs(cell_id_list):
    
    '''
    loops through a bunch of presynaptic neurons to give its synaptic outputs
    args: 
    cell_id_list = list of cell id's
    returns: 
    a big dataframe containing vertically stacked dataframes for each neuron
    '''
    
    list_of_dfs = []
    
    # looping through the list of presyn cell id's 
    for cell_id in cell_id_list:

        # get all the synapses from this cell
        out_df = client.materialize.synapse_query(pre_ids=cell_id)

        # merge this with the synapses with the neuron table
        # use how='left' to keep all the synapses but drop unused nuclei
        post_syn_df = out_df.merge(
            soma_count_df,
            left_on='post_pt_root_id',
            right_index=True,
            how='left',
        ).fillna(0)

        # filtering my df by num_soma
        # the presyn neuron here is pre_pt_root_id, so that's the same for the whole table
        # the postsyn neuron for each synapse is post_pt_root_id
        post_syn_df=post_syn_df.loc[post_syn_df['num_soma']==1.0]

        # merge in the e/i prediction into a dataframe
        merged_df = post_syn_df.merge(
                    ei_df_single[['pt_root_id', 'cell_type', 'pt_position']],
                    left_on='post_pt_root_id',
                    right_on='pt_root_id',
                    how='left',
                ).drop(columns='pt_root_id')
        
        # merge in the cell type prediction into a dataframe
        merged_df = merged_df.merge(predicted_cell_type_df[['pt_root_id', 'predicted_cell_type', 'pt_position']],
                    left_on='post_pt_root_id',
                    right_on='pt_root_id',
                    how='left',
                    suffixes=["_ei", "_ct"]
                ).drop(columns='pt_root_id')
        
        filtered_merged_df= merged_df[np.isnan(merged_df.post_pt_root_id)==0]

        # converting from voxels to microns 
        #merged_df['pt_position_'] = merged_df['pt_position']
        #merged_df['ctr_pt_position'] = merged_df['ctr_pt_position']

        # calculating synapse lengths
        syn_pos = np.array(filtered_merged_df['ctr_pt_position'])
        soma_pos = np.array(filtered_merged_df['pt_position_ei'])
        syn_distance = []
        
#         syn_distance = np.linalg.norm(syn_pos - soma_pos, axis=1)

        # to convert voxels to microns
        for i in range(len(merged_df)):
            dist = np.linalg.norm(np.array([4, 4, 40]) /1000 * (syn_pos[i]-soma_pos[i]))
            syn_distance.append(dist)
        filtered_merged_df['synapse_distance'] = syn_distance
        
        list_of_dfs.append(filtered_merged_df)
    multi_neuron_df = pd.concat(list_of_dfs)
    
    return multi_neuron_df

In[15]:

new new

In [ ]:
def get_multi_neuron_inputs(cell_id_list):
    
    '''
    loops through a bunch of postsynaptic neurons to give its synaptic inputs
    args: 
    cell_id_list = list of cell id's
    returns: 
    a big dataframe containing vertically stacked dataframes for each neuron
    '''
    
    list_of_dfs = []
    
    # looping through the list of presyn cell id's 
    for cell_id in cell_id_list:

        # get all the synapses from this cell
        in_df = client.materialize.synapse_query(post_ids=cell_id)

        # merge this with the synapses with the neuron table
        # use how='left' to keep all the synapses but drop unused nuclei
        pre_syn_df = in_df.merge(
            soma_count_df,
            left_on='pre_pt_root_id',
            right_index=True,
            how='left',
        ).fillna(0)

        # filtering my df by num_soma
       
        pre_syn_df=pre_syn_df.loc[pre_syn_df['num_soma']==1.0]

        # merge in the e/i prediction into a dataframe
        merged_df = pre_syn_df.merge(
                    ei_df_single[['pt_root_id', 'cell_type', 'pt_position']],
                    left_on='pre_pt_root_id',
                    right_on='pt_root_id',
                    how='left',
                ).drop(columns='pt_root_id')
        
        # merge in the cell type prediction into a dataframe
        merged_df = merged_df.merge(predicted_cell_type_df[['pt_root_id', 'predicted_cell_type', 'pt_position']],
                    left_on='pre_pt_root_id',
                    right_on='pt_root_id',
                    how='left',
                    suffixes=["_ei", "_ct"]
                ).drop(columns='pt_root_id')
        
        filtered_merged_df= merged_df[np.isnan(merged_df.pre_pt_root_id)==0]

In [ ]:
        # calculating synapse lengths
        syn_pos = np.array(filtered_merged_df['ctr_pt_position'])
        soma_pos = np.array(filtered_merged_df['pt_position_ei'])
        syn_distance = []
        
#         syn_distance = np.linalg.norm(syn_pos - soma_pos, axis=1)

        # to convert voxels to microns
        for i in range(len(merged_df)):
            dist = np.linalg.norm(np.array([4, 4, 40]) /1000 * (syn_pos[i]-soma_pos[i]))
            syn_distance.append(dist)
        filtered_merged_df['synapse_distance'] = syn_distance
        
        list_of_dfs.append(filtered_merged_df)
    multi_neuron_inputs_df = pd.concat(list_of_dfs)
    
    return multi_neuron_inputs_df

In[31]:

In [ ]:
def get_inputs_df(cell_id_list, cell_type_ei):
    
    '''
    loops through a bunch of postsynaptic neurons to give its synaptic inputs
    args: 
    cell_id_list = list of cell id's
    cell_type_ei = 'excitatory' or 'inhibitory'
    returns: 
    a big dataframe containing vertically stacked dataframes for each neuron
    '''
    
    list_of_dfs = []
    
    # looping through the list of presyn cell id's 
    for cell_id in cell_id_list:

        # get all the synapses from this cell
        in_df = client.materialize.synapse_query(post_ids=cell_id)

        # merge this with the synapses with the neuron table
        # use how='left' to keep all the synapses but drop unused nuclei
        pre_syn_df = in_df.merge(
            soma_count_df,
            left_on='pre_pt_root_id',
            right_index=True,
            how='left',
        ).fillna(0)

        # filtering my df by num_soma
        
        pre_syn_df=pre_syn_df.loc[pre_syn_df['num_soma']==1.0]

        # merge in the e/i prediction into a dataframe
        merged_df = pre_syn_df.merge(
                    ei_df_single[['pt_root_id', 'cell_type', 'pt_position']],
                    left_on='pre_pt_root_id',
                    right_on='pt_root_id',
                    how='left',
                ).drop(columns='pt_root_id')
        
        # merge in the cell type prediction into a dataframe
        merged_df = merged_df.merge(predicted_cell_type_df[['pt_root_id', 'predicted_cell_type', 'pt_position']],
                    left_on='pre_pt_root_id',
                    right_on='pt_root_id',
                    how='left',
                    suffixes=["_ei", "_ct"]
                ).drop(columns='pt_root_id')

        # calculating synapse lengths
        syn_pos = np.array(merged_df['ctr_pt_position'])
        soma_pos = np.array(merged_df['pt_position_ei'])
        syn_distance = []
        
#         syn_distance = np.linalg.norm(syn_pos - soma_pos, axis=1)

        # to convert voxels to microns
        for i in range(len(merged_df)):
            dist = np.linalg.norm(np.array([4, 4, 40]) /1000 * (syn_pos[i]-soma_pos[i]))
            syn_distance.append(dist)
        merged_df['synapse_distance'] = syn_distance
        
        inputs_df = merged_df.loc[merged_df['cell_type']==cell_type_ei]
        list_of_dfs.append(inputs_df)
    multi_neuron_inputs_df = pd.concat(list_of_dfs)
    
    return multi_neuron_inputs_df

In[16]:

In [ ]:
def get_outputs_df(cell_id_list, cell_type_ei):
    
    '''
    loops through a bunch of presynaptic neurons to give its synaptic outputs
    args: 
    cell_id_list = list of cell id's
    cell_type_ei = 'excitatory' or 'inhibitory'
    returns: 
    a big dataframe containing vertically stacked dataframes for each neuron
    '''
    
    list_of_dfs = []
    
    # looping through the list of presyn cell id's 
    for cell_id in cell_id_list:

        # get all the synapses from this cell
        out_df = client.materialize.synapse_query(pre_ids=cell_id)

        # merge this with the synapses with the neuron table
        # use how='left' to keep all the synapses but drop unused nuclei
        post_syn_df = out_df.merge(
            soma_count_df,
            left_on='post_pt_root_id',
            right_index=True,
            how='left',
        ).fillna(0)

        # filtering my df by num_soma
        # the presyn neuron here is pre_pt_root_id, so that's the same for the whole table
        # the postsyn neuron for each synapse is post_pt_root_id
        post_syn_df=post_syn_df.loc[post_syn_df['num_soma']==1.0]

        # merge in the e/i prediction into a dataframe
        merged_df = post_syn_df.merge(
                    ei_df_single[['pt_root_id', 'cell_type', 'pt_position']],
                    left_on='post_pt_root_id',
                    right_on='pt_root_id',
                    how='left',
                ).drop(columns='pt_root_id')
        
        # merge in the cell type prediction into a dataframe
        merged_df = merged_df.merge(predicted_cell_type_df[['pt_root_id', 'predicted_cell_type', 'pt_position']],
                    left_on='post_pt_root_id',
                    right_on='pt_root_id',
                    how='left',
                    suffixes=["_ei", "_ct"]
                ).drop(columns='pt_root_id')

        # calculating synapse lengths
        syn_pos = np.array(merged_df['ctr_pt_position'])
        soma_pos = np.array(merged_df['pt_position_ei'])
        syn_distance = []
        
#         syn_distance = np.linalg.norm(syn_pos - soma_pos, axis=1)

        # to convert voxels to microns
        for i in range(len(merged_df)):
            dist = np.linalg.norm(np.array([4, 4, 40]) /1000 * (syn_pos[i]-soma_pos[i]))
            syn_distance.append(dist)
        merged_df['synapse_distance'] = syn_distance
        
        outputs_df = merged_df.loc[merged_df['cell_type']==cell_type_ei]
        list_of_dfs.append(outputs_df)
    multi_neuron_df = pd.concat(list_of_dfs)
    
    return multi_neuron_df

In[17]:

In [ ]:
inh_outputs_df = ei_df_single.loc[ei_df_single['cell_type']=='inhibitory']
exc_outputs_df = ei_df_single.loc[ei_df_single['cell_type']=='excitatory']

In [ ]:
inh_ids = inh_outputs_df['pt_root_id']
exc_ids = exc_outputs_df['pt_root_id']

In [ ]:
inh_id_list = inh_ids.to_list()[:50]
exc_id_list = exc_ids.to_list()[:50]

In[18]:

In [ ]:
multi_inh_df = get_multi_neuron_outputs(inh_id_list)

In [ ]:
sns.distplot(multi_inh_df['synapse_distance'], hist=True, kde=False, 
             bins=100, color = 'blue',
             hist_kws={'edgecolor':'black'})
# Add labels
plt.title('Histogram of all postsynaptic inhibitory synapses by distance from soma')
plt.xlabel('distance from soma (microns)')
plt.ylabel('# of inhibitory synapses');
plt.axis([0, 200, 0, 2500]);

In[19]:

In [ ]:
multi_exc_df = get_multi_neuron_outputs(exc_id_list)

In [ ]:
sns.distplot(multi_exc_df['synapse_distance'], hist=True, kde=False, 
             bins=100, color = 'blue',
             hist_kws={'edgecolor':'black'})
# Add labels
plt.title('Histogram of all postsynaptic excitatory synapses by distance from soma')
plt.xlabel('distance from soma (microns)')
plt.ylabel('# of excitatory synapses');
plt.axis([0, 200, 0, 2500])

In[20]:

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))

In [ ]:
sns.histplot(multi_inh_df['synapse_distance'], bins=100, label='inhibitory', color='blue', alpha=0.5)
sns.histplot(multi_exc_df['synapse_distance'], bins=100, label='excitatory', color='red', alpha=0.5);

In [ ]:
plt.title('Postsynaptic synapses from I vs E cells by distance from soma')
plt.xlabel('distance from soma (microns)')
plt.ylabel('# of synapses');
plt.legend(labels= ['inhibitory', 'excitatory']);
plt.savefig('ei_outputs.png')

In[21]:

In [ ]:
multi_inh_df.predicted_cell_type.value_counts()

In[22]:

In [ ]:
vip_to_exc = get_outputs_df(vip_ids, 'excitatory')
vip_to_inh = get_outputs_df(vip_ids, 'inhibitory')

In[23]:

In [ ]:
vip_to_inh.reset_index()

In[ ]:

In[24]:

In [ ]:
fig, ax = plt.subplots(figsize=(10,8))

In [ ]:
sns.histplot(vip_to_exc.reset_index(), x='synapse_distance', bins=np.linspace(0, 500, 100), color='blue', alpha=0.5, stat='probability')
sns.histplot(vip_to_inh.reset_index(), x='synapse_distance', bins=np.linspace(0, 500, 100), color='gray', alpha=0.5, stat='probability')

ns.histplot(vip_to_inh['synapse_distance'], x=np.linspace(0, 500), bins=np.linspace(0, 500, 100), color='blue', fill=False, weights=np.ones(len(vip_to_inh)) / len(vip_to_inh))

In [ ]:
plt.title('Postsynaptic outputs from BPC')
plt.xlabel('distance from soma (microns)')
plt.ylabel('proportion of synapses');
plt.legend(labels= ['BPC to excitatory', 'BPC to inhibitory']);
plt.savefig('bpc.png')

In[25]:

In [ ]:
pv_to_exc = get_outputs_df(pv_ids, 'excitatory')
pv_to_inh = get_outputs_df(pv_ids, 'inhibitory')

In[26]:

In [ ]:
pv_to_exc.head()

In[27]:

In [ ]:
fig, ax = plt.subplots(figsize=(10,8))

In [ ]:
sns.histplot(pv_to_exc.reset_index(), x='synapse_distance', bins=np.linspace(0, 300, 100), color='red', alpha=0.5, stat='probability')
sns.histplot(pv_to_inh.reset_index(), x='synapse_distance', bins=np.linspace(0, 300, 100), color='gray', alpha=0.5, stat='probability')

In [ ]:
plt.title('Postsynaptic outputs from BC')
plt.xlabel('distance from soma (microns)')
plt.ylabel('proportion of synapses');
plt.legend(labels= ['BC to excitatory', 'BC to inhibitory']);
plt.savefig('bc.png')

In[28]:

In [ ]:
sst_to_exc = get_outputs_df(sst_ids, 'excitatory')
sst_to_inh = get_outputs_df(sst_ids, 'inhibitory')

In[29]:

In [ ]:
fig, ax = plt.subplots(figsize=(10,8))

In [ ]:
sns.histplot(sst_to_exc.reset_index(), x='synapse_distance', bins=np.linspace(0, 500, 100), color='green', alpha=0.5, stat='probability')
sns.histplot(sst_to_inh.reset_index(), x='synapse_distance', bins=np.linspace(0, 500, 100), color='gray', alpha=0.5, stat='probability')

In [ ]:
plt.title('Postsynaptic outputs from MC')
plt.xlabel('distance from soma (microns)')
plt.ylabel('proportion of synapses');
plt.legend(labels= ['MC to excitatory', 'MC to inhibitory']);
plt.savefig('mc.png')

In[32]:

In [ ]:
exc_inputs = get_inputs_df(functional_ids, 'excitatory')
inh_inputs = get_inputs_df(functional_ids, 'inhibitory')

In[ ]:

In [ ]:
fig, ax = plt.subplots(figsize=(12,8))

In [ ]:
sns.histplot(exc_inputs['synapse_distance'], bins=100, label='excitatory', color='red', alpha=0.5, stat='probability')
sns.histplot(inh_inputs['synapse_distance'], bins=100, label='inhibitory', color='blue', alpha=0.5, stat='probability')

In [ ]:
plt.title('Synaptic inputs onto functionally coregistered neurons')
plt.xlabel('distance from soma (microns)')
plt.ylabel('proportion of synapses');
plt.legend(labels= ['excitatory inputs', 'inhibitory inputs']);
plt.savefig('inputs_to_func_coreg.png')

In[ ]:

exc_to_single_exc = get_single_neuron_inputs(864691135181741826, 'excitatory')<br>
inh_to_single_exc = get_single_neuron_inputs(864691135181741826, 'inhibitory')

In[38]:

In [ ]:
excitatory_ids= np.array(ei_df_single['pt_root_id'])
cln_ids = np.array(cln_df['pt_root_id'])

In [ ]:
cln_exc_ids = np.intersect1d(excitatory_ids, cln_ids).tolist()
len(cln_exc_ids)

In[35]:

In [ ]:
inh_perisom_inputs = []
for exc_id in cln_exc_ids:
    inh_to_single_exc = get_single_neuron_inputs(exc_id, 'inhibitory')
    
    perisom_inh_inputs = inh_to_single_exc.loc[inh_to_single_exc['synapse_distance']<50]
    num_perisom_inputs = len(perisom_inh_inputs)
    inh_perisom_inputs.append(num_perisom_inputs)

In[ ]:

In [ ]:
len(inh_perisom_inputs)

In[ ]:

In [ ]:
inh_perisom_inputs

In[36]:

In [ ]:
inh_perisom_inputs = [15,
5,
10,
9,
4,
3,
24,
4,
16,
20,
14,
34,
5,
20,
12,
0,
8,
5,
5,
40,
17,
27,
13,
6,
15,
16,
5,
54,
10,
9,
8,
9,
5,
9,
3,
10,
12,
7,
5,
27,
30,
13,
12,
5,
2,
5,
23,
19,
34,
19,
44,
3,
13,
32,
8,
17,
8,
14,
20,
20,
21,
6,
28,
10,
38,
10,
5,
6,
8,
8,
1,
25,
5,
12,
19,
2,
2,
15,
15,
21,
35,
8,
21,
9,
12,
9,
36,
13,
3,
6,
7,
20,
6,
7,
9,
10,
10,
28,
36,
4,
5,
10,
13,
44,
21,
31,
16,
11,
29,
8,
18,
13,
21,
10,
2,
27,
4,
31,
3,
6,
8,
21,
11,
12,
7,
1,
12,
29,
13,
8,
12,
5,
7,
0,
7,
9,
23,
13,
10,
8,
30,
10,
46,
13,
7,
5,
31,
0,
4,
7,
3,
14,
0,
36,
11,
15,
3,
40,
16,
19,
0,
33,
1,
34,
4,
15,
9,
5,
1,
13,
10,
36,
8,
10,
15,
22,
16,
12,
21,
8,
14,
5,
2,
9,
4,
8,
26,
4,
23,
3,
8,
8,
11,
23,
15,
18,
5,
19,
12,
12,
4,
12,
14,
7,
14,
6,
1,
4,
1,
9,
10,
0,
7,
11,
4,
19,
0,
8,
5,
4,
6,
10,
16,
15,
31,
11,
5,
23,
9,
23,
9,
17,
4,
8,
12,
10,
2,
5,
6,
21,
25,
28,
13]

In[37]:

In [ ]:
len(inh_perisom_inputs)

In[52]:

In [ ]:
corr_merged_df

In[53]:

In [ ]:
metric_df = pd.DataFrame({'pt_root_id':cln_exc_ids, 'perisom_inputs':inh_perisom_inputs})

In[57]:

In [ ]:
metric_df

In[71]:

In [ ]:
metric_merged_df = metric_df.merge(corr_merged_df, on=['pt_root_id'], how='inner')
metric_merged_df 

In[66]:

In [ ]:
pupil_corr = metric_merged_df['pupil_spike_corr_r']
perisom_metric = metric_merged_df['perisom_inputs']

In [ ]:
plt.scatter(pupil_corr, perisom_metric)

In[100]:

In [ ]:
metric_df_upper_layers = metric_merged_df.loc[metric_merged_df['um_z']<280]

In[101]:

In [ ]:
metric_df_upper_layers

In[104]:

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))

In [ ]:
pupil_corr = metric_df_upper_layers['pupil_spike_corr_r']
perisom_metric = metric_df_upper_layers['perisom_inputs']

In [ ]:
plt.scatter(pupil_corr, perisom_metric)
plt.xlabel('pupil spike correlation r')
plt.ylabel('number of perisomatic inputs (< 50 um)');
plt.savefig('corr.png')

In[95]:

In [ ]:
import seaborn as sns
from scipy import stats

In[94]:

In [ ]:
sns.scatterplot(data=metric_merged_df, x="perisom_inputs", y="pupil_spike_corr_r", 
                hue="um_z", legend=False, palette="spring")

In[96]:

In [ ]:
stats.pearsonr(metric_merged_df["perisom_inputs"], metric_merged_df["pupil_spike_corr_r"])

In[ ]:

fix this to be vip vs pv inputs onto an E cell

Lets make a matrix of synapse positions for this neuron<br>
and plot their positions in x,y along with the soma position as a red circle

In [ ]:
exc_syn_pos_nm = convert_to_nm(exc_to_single_exc.ctr_pt_position)
inh_syn_pos_nm = convert_to_nm(inh_to_single_exc.ctr_pt_position)

In [ ]:
f ,ax =plt.subplots(1,2,figsize=(12,6))
ax[0].scatter(exc_syn_pos_nm[:,0]/1000, exc_syn_pos_nm[:,1]/1000, c='r')
ax[0].scatter(inh_syn_pos_nm[:,0]/1000, inh_syn_pos_nm[:,1]/1000, c='b')

add the soma position as a red circle

In [ ]:
soma_pos = convert_to_nm(clean_dend_df.pt_position)
ax[0].scatter([soma_pos[0,0]/1000],[soma_pos[0,1]/1000],c='y',s=150)
ax[0].set_aspect('equal')
ax[0].set_xlabel('x (um)')
ax[0].set_ylabel('y (um)')
ax[0].set_ylim(1100,300)

In [ ]:
ax[1].scatter(exc_syn_pos_nm[:,0]/1000, exc_syn_pos_nm[:,2]/1000, c='r')
ax[1].scatter(inh_syn_pos_nm[:,0]/1000, inh_syn_pos_nm[:,1]/1000, c='b')

add the soma position as a red circle

In [ ]:
soma_pos = convert_to_nm(clean_dend_df.pt_position)
ax[1].scatter([soma_pos[0,0]/1000],[soma_pos[0,2]/1000],c='y',s=150)
ax[1].set_aspect('equal')
ax[1].set_xlabel('x (um)')
ax[1].set_ylabel('z (um)')

In[ ]:

In[ ]:

In [ ]:
vip_syn_pos_nm = convert_to_nm(vip_to_exc.ctr_pt_position)

In [ ]:
f ,ax =plt.subplots(1,2,figsize=(12,6))
ax[0].scatter(vip_syn_pos_nm[:,0]/1000, vip_syn_pos_nm[:,1]/1000, c='r')
#ax[0].scatter(inh_syn_pos_nm[:,0]/1000, inh_syn_pos_nm[:,1]/1000, c='b')

add the soma position as a red circle

In [ ]:
soma_pos = convert_to_nm(clean_dend_df.pt_position)
ax[0].scatter([soma_pos[0,0]/1000],[soma_pos[0,1]/1000],c='g',s=150)
ax[0].set_aspect('equal')
ax[0].set_xlabel('x (um)')
ax[0].set_ylabel('y (um)')
ax[0].set_ylim(1100,300)

In [ ]:
ax[1].scatter(vip_syn_pos_nm[:,0]/1000, vip_syn_pos_nm[:,2]/1000, c='r')
#ax[1].scatter(inh_syn_pos_nm[:,0]/1000, inh_syn_pos_nm[:,1]/1000, c='b')

add the soma position as a red circle

In [ ]:
soma_pos = convert_to_nm(clean_dend_df.pt_position)
ax[1].scatter([soma_pos[0,0]/1000],[soma_pos[0,2]/1000],c='y',s=150)
ax[1].set_aspect('equal')
ax[1].set_xlabel('x (um)')
ax[1].set_ylabel('z (um)')

In[ ]:

In [ ]:
np.array(inh_to_single['predicted_cell_type'])

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In [ ]:
single_cell_id = vip_ids[4]

In [ ]:
out_df = client.materialize.synapse_query(pre_ids=single_cell_id)

In [ ]:
post_syn_df = out_df.merge(
            soma_count_df,
            left_on='post_pt_root_id',
            right_index=True,
            how='left',
        ).fillna(0)

In [ ]:
post_syn_df=post_syn_df.loc[post_syn_df['num_soma']==1.0]

In [ ]:
merged_df = post_syn_df.merge(
                    ei_df_single[['pt_root_id', 'cell_type', 'pt_position']],
                    left_on='post_pt_root_id',
                    right_on='pt_root_id',
                    how='left',
                ).drop(columns='pt_root_id')
        
merged_df = merged_df.merge(predicted_cell_type_df[['pt_root_id', 'predicted_cell_type', 'pt_position']],
                    left_on='post_pt_root_id',
                    right_on='pt_root_id',
                    how='left',
                    suffixes=["_ei", "_ct"]
                ).drop(columns='pt_root_id')

In [ ]:
syn_pos = np.array(merged_df['ctr_pt_position'])
soma_pos = np.array(merged_df['pt_position_ei'])
syn_distance = []
        
    # to convert voxels to microns
for i in range(len(merged_df)):
    dist = np.linalg.norm(np.array([4, 4, 40]) /1000 * (syn_pos[i]-soma_pos[i]))
    syn_distance.append(dist)
merged_df['synapse_distance'] = syn_distance

In [ ]:
single_neuron_to_exc = merged_df.loc[merged_df['cell_type']=='excitatory']
single_neuron_to_inh = merged_df.loc[merged_df['cell_type']=='inhibitory']

In[ ]:

In [ ]:
merged_df

In[ ]:

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))

In [ ]:
sns.histplot(single_neuron_to_exc['synapse_distance'], bins=100, color='pink')
sns.histplot(single_neuron_to_inh['synapse_distance'], bins=100, color='blue')

In [ ]:
plt.title('Postsynaptic synapses from one neuron by distance from soma')
plt.xlabel('distance from soma (microns)')
plt.ylabel('# of synapses');
plt.legend(labels= ['neuron_to_exc', 'neuron_to_inh']);

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

i think all this shit is wrong but we might be able to use? 

In[ ]:

In [ ]:
multi_inh_ct_df = multi_inh_df[(multi_inh_df['predicted_cell_type']=='BC') |
                               (multi_inh_df['predicted_cell_type']=='BPC') |
                               (multi_inh_df['predicted_cell_type']=='BP')]

In [ ]:
multi_exc_ct_df = multi_exc_df[(multi_exc_df['predicted_cell_type']=='BC') |
                               (multi_exc_df['predicted_cell_type']=='BPC') |
                               (multi_exc_df['predicted_cell_type']=='BP')]

In[ ]:

In [ ]:
pv_inh = multi_inh_ct_df.loc[multi_inh_ct_df['predicted_cell_type']=='BC']
vip_inh = multi_inh_ct_df.loc[multi_inh_ct_df['predicted_cell_type']=='BPC']
sst_inh = multi_inh_ct_df.loc[multi_inh_ct_df['predicted_cell_type']=='BP']

In[ ]:

In [ ]:
pv_inh = multi_inh_ct_df.loc[multi_inh_ct_df['predicted_cell_type']=='BC']
vip_inh = multi_inh_ct_df.loc[multi_inh_ct_df['predicted_cell_type']=='BPC']
sst_inh = multi_inh_ct_df.loc[multi_inh_ct_df['predicted_cell_type']=='BP']

In[ ]:

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))

In [ ]:
sns.histplot(pv_inh['synapse_distance'], bins=100, label='pv_inh', color='blue')
sns.histplot(vip_inh['synapse_distance'], bins=100, label='vip_inh', color='pink')
sns.histplot(sst_inh['synapse_distance'], bins=100, label='sst_inh', color='yellow');

In [ ]:
plt.title('Postsynaptic synapses from inhibitory cells by distance from soma')
plt.xlabel('distance from soma (microns)')
plt.ylabel('# of synapses');
plt.legend(labels= ['inhibitory', 'excitatory']);

In[ ]:

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))

In [ ]:
sns.histplot(pv_exc['synapse_distance'], bins=100, label='pv_exc', color='blue')
sns.histplot(vip_exc['synapse_distance'], bins=100, label='vip_exc', color='pink')
sns.histplot(sst_exc['synapse_distance'], bins=100, label='sst_exc', color='yellow')

In[ ]:

In[ ]: